In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, KFold, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_squared_error

In [2]:
df = pd.read_csv("cleaned_chronic_data.csv")
df.head()

,BENE_SEX_IDENT_CD,BENE_AGE_CAT_CD,CC_ALZHDMTA,CC_CANCER,CC_CHF,CC_CHRNKIDN,CC_COPD,CC_DEPRESSN,CC_DIABETES,CC_ISCHMCHT,...,AVE_OP_PAY_PB_EQ_12,AVE_OTH_PAY_PB_EQ_12,AVE_CA_VST_PB_EQ_12,AVE_OP_VST_PB_EQ_12,BENE_COUNT_PD_EQ_12,AVE_PDE_CST_PD_EQ_12,AVE_PDE_PD_EQ_12,Plan_A_Total_Cost,Plan_B_Total_Cost,Plan_D_Total_Cost
0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,278.0,139.0,1.948,1.127,516476.0,2480.0,29.415,290.0,890.0,2480.0
1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,564.0,295.0,2.957,2.786,753223.0,3479.0,28.795,913.0,1688.0,3479.0
2,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,980.0,444.0,6.070,3.872,1685.0,2751.0,33.657,3536.0,3064.0,2751.0
3,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1753.0,860.0,6.105,6.118,3963.0,3979.0,43.839,7164.0,5242.0,3979.0
4,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,851.0,316.0,7.099,3.109,16427.0,2876.0,34.996,1487.0,3145.0,2876.0


First we isolate the features we want: age, sex, conditions, dual eligibility, but we will drop the multiple condition column (as it is dependent on other columns). We then make a label column.

In [3]:
X = df.iloc[:,0:15]
X = X.drop(X.columns[13], axis=1).values
X

array([[1., 1., 0., ..., 0., 0., 0.],
       [1., 1., 0., ..., 0., 0., 1.],
       [1., 1., 0., ..., 0., 1., 0.],
       ...,
       [2., 6., 1., ..., 1., 0., 1.],
       [2., 6., 1., ..., 0., 0., 1.],
       [2., 6., 1., ..., 1., 0., 1.]])

In [4]:
Y = df["Plan_D_Total_Cost"].values
Y

array([2480., 3479., 2751., ..., 6917., 6945., 8171.])

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=0)

In [6]:
#Fitting the model, a first quick test

regr = LinearRegression()
regr.fit (X_train, Y_train)

# The coefficients
print ('Coefficients: ', regr.coef_)

Coefficients:  [ 131.49575344 -510.27134982 1039.7212298   429.80809546  444.52306368
  544.67670825  837.74132836  961.1277556  1055.56357961  396.43327908
  654.97863105  380.13270836 -139.5681478  1852.96636268]


In [7]:
#Predictions

Y_hat= regr.predict(X_test)

print("Residual sum of squares: %.2f"
      % np.mean((Y_hat - Y_test) ** 2))

print('Variance score: %.5f' % regr.score(X_test, Y_test))

Residual sum of squares: 832769.79
Variance score: 0.78496


This syncs up with what our pipeline said we should get. Now we will run through several models.

In [8]:
models = [('Linear Regression', LinearRegression()),('Gradient Boosting', GradientBoostingRegressor()), ('Random Forest', RandomForestRegressor())]

In [9]:
for model in models:
    reg=model[1]
    reg.fit(X_train,Y_train)
    prediction = reg.predict(X_test)

    print(model[0])
    print('R2: ', r2_score(Y_test, prediction))
    print('RMSE: ', np.sqrt(mean_squared_error(Y_test, prediction)))
    print('-'*30)

    #model[2]_regr = (model[0], r2_score(Y_test, prediction), np.sqrt(mean_squared_error(Y_test, prediction)))

Linear Regression
R2:  0.7849608589110498
RMSE:  912.5622139032246
------------------------------
Gradient Boosting
R2:  0.8792003077122509
RMSE:  683.969345631743
------------------------------
Random Forest
R2:  0.8715928063937564
RMSE:  705.1774254918869
------------------------------


In [12]:
#We will save the output to make a table later!
lin_reg = ('Linear Reg', 0.7849608589110498, 912.5622139032246)
grad_boost = ('Gradient Boosting', 0.8792003077122509, 683.969345631743)

Gradiant boosting looks to be the best from that batch. Now we will look at KNN, Decision Trees, and Random Forests, but this time with some hyperparameter tuning.

In [10]:
params ={'n_neighbors' : [6, 7, 8, 9, 10, 11, 12], 'weights': ['distance'], 'p':[1,2]}
knn=KNeighborsRegressor()
gs=GridSearchCV(estimator=knn, param_grid=params, cv=10, n_jobs=-1, scoring='neg_mean_squared_error')
gs.fit(X_train,Y_train)
print(gs.best_estimator_)
knn=gs.best_estimator_
knn.fit(X_train, Y_train)
prediction=knn.predict(X_test)

print('-'*30)
r2=r2_score(Y_test,prediction)
print('R2: ', r2)
rmse = np.sqrt(mean_squared_error(Y_test, prediction))
print('RMSE: ', rmse)

kn = ('KNN', r2, rmse)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=11, p=1,
                    weights='distance')
------------------------------
R2:  0.8576041525998331
RMSE:  742.5956245844387


In [11]:
params = {'criterion': ['mse'],
     'splitter': ['best', 'random'],
     'max_depth': [2*n for n in range(1,10)],
     'max_features': ['auto', 'sqrt'],
     'min_samples_leaf': [1, 2, 4],
     'min_samples_split': [2, 5, 10]}

tree = DecisionTreeRegressor()
gs = GridSearchCV(estimator=tree, param_grid=params, cv=5, n_jobs=-1, scoring='neg_mean_squared_error')
gs.fit(X_train, Y_train)
print(gs.best_estimator_)

tree = gs.best_estimator_
tree.fit(X_train, Y_train)
prediction=tree.predict(X_test)

print('-'*30)
r2=r2_score(Y_test,prediction)
print('R2: ', r2)
rmse = np.sqrt(mean_squared_error(Y_test, prediction))
print('RMSE: ', rmse)

dec_tree = ('Decision Tree', r2, rmse)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=16,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=4, min_samples_split=10,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='random')
------------------------------
R2:  0.8400275360432958
RMSE:  787.0935270050246


In [13]:
params = {'n_estimators': [1000], 'max_depth': [7,8,9], 'max_features':['auto','sqrt'], 'min_samples_leaf':[3,4,5], 'min_samples_split':[0.01]}
rfor=RandomForestRegressor()
gs=GridSearchCV(estimator=rfor,param_grid=params,cv=5, n_jobs=-1, scoring='neg_mean_squared_error')
gs.fit(X_train, Y_train)
print(gs.best_estimator_)

rfor=gs.best_estimator_
rfor.fit(X_train, Y_train)
prediction = rfor.predict(X_test)

print('-'*30)
r2=r2_score(Y_test,prediction)
print('R2: ', r2)
rmse = np.sqrt(mean_squared_error(Y_test, prediction))
print('RMSE: ', rmse)

rforest = ('Random Forest', r2, rmse)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=9, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=4,
                      min_samples_split=0.01, min_weight_fraction_leaf=0.0,
                      n_estimators=1000, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)
------------------------------
R2:  0.8032588552383169
RMSE:  872.8735243004098


In [14]:
results = pd.DataFrame([lin_reg,grad_boost,kn,dec_tree,rforest], columns=['model','R2','RMSE'])
results.sort_values('R2', ascending=False)

,model,R2,RMSE
1,Gradient Boosting,0.879200,683.969346
2,KNN,0.857604,742.595625
3,Decision Tree,0.840028,787.093527
4,Random Forest,0.803259,872.873524
0,Linear Reg,0.784961,912.562214


Looking at the above outputs, we see that the Gradient Boosting model is the best (highest R2, low RMSE, quick computation time). As a sanity check, we will also run the data through Azure's AML and see what that claims the best model is!

We will make a new data asset in our blob storage for this task:

In [ ]:
chronic_aml_table = df.iloc[:,0:15]
chronic_aml_table['Drug_Cost']=df['Plan_D_Total_Cost']
chronic_aml_table = chronic_aml_table.drop(chronic_aml_table.columns[13], axis=1)
chronic_aml_table.head()

In [16]:
chronic_aml_table.to_csv("chronic_aml_data.csv", index=False)

In [17]:
#Connect to workspace
subscription_id = "<MASKED>"
resource_group = "<MASKED>"
workspace = "<MASKED>"

from azure.ai.ml import MLClient
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes
from azure.identity import DefaultAzureCredential

# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

In [ ]:
path = "azureml://<MASKED>/chronic_aml_data.csv"
my_data = Data(
    path=path,
    type=AssetTypes.URI_FILE,
    description="Data for aml",
    name="chronic-aml-data",
    version="1",
)

ml_client.data.create_or_update(my_data)